In [1]:
import plotly.graph_objects as go

from datetime import datetime
import sys


In [2]:
# Ralative imports
# Add the src directory to the Python path
sys.path.insert(0, '../src')

from botcoin.data.historical import YfDataManager
from botcoin.utils.stream_data import generate_price_stream

In [3]:
hdm = YfDataManager(ticker="AAPL")

# Create a naive datetime
start   = datetime(year=2025, month=4, day=15, hour=0, minute=0, second=0)
end     = datetime(year=2025, month=4, day=16, hour=0, minute=0, second=0)

start   = hdm.tz.localize(start)
end     = hdm.tz.localize(end)

In [4]:
hdm.check_data_in_local(start, end)

False

In [5]:
df = hdm.get_data(start=start, end=end)

2025-04-26 11:40:31,317 - YfDataManager - INFO: Fetching data from Yahoo Finance.
2025-04-26 11:40:31,319 - YfDataManager - INFO: Fetching data from 2025-04-15 00:00:00-04:00 to 2025-04-16 00:00:00-04:00
2025-04-26 11:40:31,320 - yfinance - DEBUG: Entering download()
2025-04-26 11:40:31,322 - yfinance - DEBUG: Disabling multithreading because DEBUG logging enabled
2025-04-26 11:40:31,323 - yfinance - DEBUG: Using User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 14_7_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3 Safari/605.1.15
2025-04-26 11:40:31,326 - yfinance - DEBUG:  Entering history()
2025-04-26 11:40:31,344 - peewee - DEBUG: ('CREATE TABLE IF NOT EXISTS "_kv" ("key" VARCHAR(255) NOT NULL PRIMARY KEY, "value" VARCHAR(255)) WITHOUT ROWID', [])
2025-04-26 11:40:31,345 - peewee - DEBUG: ('SELECT "t1"."key", "t1"."value" FROM "_kv" AS "t1" WHERE ("t1"."key" = ?) LIMIT ? OFFSET ?', ['AAPL', 1, 0])
2025-04-26 11:40:31,348 - yfinance - DEBUG:   Entering history()
2025-04-26 

YF.download() has changed argument auto_adjust default to True


2025-04-26 11:40:31,594 - urllib3.connectionpool - DEBUG: https://query1.finance.yahoo.com:443 "GET /v1/test/getcrumb HTTP/1.1" 200 11
2025-04-26 11:40:31,597 - yfinance - DEBUG: crumb = 'zxt6Ow5CAAQ'
2025-04-26 11:40:31,598 - yfinance - DEBUG:       Exiting _get_cookie_and_crumb_basic()
2025-04-26 11:40:31,599 - yfinance - DEBUG:      Exiting _get_cookie_and_crumb()
2025-04-26 11:40:31,604 - urllib3.connectionpool - DEBUG: Starting new HTTPS connection (1): query2.finance.yahoo.com:443
2025-04-26 11:40:31,730 - urllib3.connectionpool - DEBUG: https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/AAPL?period1=1744689600&period2=1744776000&interval=1m&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=zxt6Ow5CAAQ HTTP/1.1" 200 None
2025-04-26 11:40:31,733 - yfinance - DEBUG: response code=200
2025-04-26 11:40:31,735 - yfinance - DEBUG:     Exiting _make_request()
2025-04-26 11:40:31,736 - yfinance - DEBUG:    Exiting get()
2025-04-26 11:40:31,741 - yfinance - DEBUG: AAP

In [6]:
df.head()

,Close,High,Low,Open,Volume
Datetime,,,,,
2025-04-15 09:30:00-04:00,202.694595,202.869995,201.789993,201.854996,2412796
2025-04-15 09:31:00-04:00,202.744995,202.750000,202.160004,202.649994,312062
2025-04-15 09:32:00-04:00,202.294998,202.800003,202.070007,202.786804,263184
2025-04-15 09:33:00-04:00,202.339996,203.240005,202.309998,202.309998,399844
2025-04-15 09:34:00-04:00,202.547302,202.850006,202.033203,202.320007,296751


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 379 entries, 2025-04-15 09:30:00-04:00 to 2025-04-15 15:59:00-04:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   379 non-null    float64
 1   High    379 non-null    float64
 2   Low     379 non-null    float64
 3   Open    379 non-null    float64
 4   Volume  379 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 17.8 KB


In [8]:
prices = generate_price_stream(df, candle_duration='1min', avg_freq_per_minute=10)

In [9]:
def plot_candles_with_price_stream(ohlc_df, price_stream_df):
    """
    Plot OHLC candlesticks and overlay price stream using Plotly.
    
    Args:
        ohlc_df (pd.DataFrame): Must have DatetimeIndex and columns ['Open', 'High', 'Low', 'Close'].
        price_stream_df (pd.DataFrame): Must have DatetimeIndex and column ['price'].
    """
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=ohlc_df.index,
        open=ohlc_df['Open'],
        high=ohlc_df['High'],
        low=ohlc_df['Low'],
        close=ohlc_df['Close'],
        name='Candles',
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

    # Simulated price points as a line
    fig.add_trace(go.Scatter(
        x=price_stream_df.index,
        y=price_stream_df['price'],
        mode='lines+markers',
        name='Simulated Prices',
        line=dict(color='blue', width=1),
        marker=dict(size=4)
    ))

    # Styling
    fig.update_layout(
        title="OHLC Candles with Simulated Tick Data",
        xaxis_title="Time",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        height=600,
        width=1000
    )

    fig.show()

In [10]:
plot_candles_with_price_stream(df, prices)